In [75]:
import pandas as pd
from collections import defaultdict
from functools import reduce

In [76]:

# Pfad zur Tabelle
table_path = '../data/mRNP_chrono/mRNP_chronology_hsa_HeLa-v1.xlsx'

# Tabelle mit pandas parsen
df = pd.read_excel(table_path, sheet_name='Sheet1')

In [77]:

# Ausgabe der Tabelle
print(df.shape)
print(df.columns)
print(df.iloc[1:10, :])
print(df[['MasterAccession', 'Primary gene name', 'BioGrid_interactor_#exp_#pub']].head(2))


(1035, 35)
Index(['MasterAccession', 'cluster', 'Primary gene name', 'Gene Names',
       'Cluster membership (corr)', 'Maximal time', 'Non-master Accessions',
       'Non-master Gene Names', 'Peak_broadness', 'Enriched_over_NoCL',
       'Over_quantified_cutoff', 'In_previous_study',
       'Not_histone_keratin_trypsin', 'Filtered_for_clustering',
       'Ribosomal protein', 'Ribosome or mitochondria related blacklist',
       'Mito-matrix and not cyto', '0m_zscore', '15m_zscore', '30m_zscore',
       '45m_zscore', '1h_zscore', '1.5h_zscore', '2h_zscore', '3h_zscore',
       '4h_zscore', '5h_zscore', 'local_HPA', 'Protein names', 'local_HCM_NMF',
       'local_HCM_SAFE', 'BioGrid_interactor_#exp_#pub',
       'Expected maximal time', 'Expected cluster',
       'RMSE between expected and observed'],
      dtype='object')
  MasterAccession cluster Primary gene name              Gene Names  \
1          P22087       I               FBL           FBL FIB1 FLRN   
2          O00267       I

In [78]:
# df[['MasterAccession', 'Primary gene name', 'BioGrid_interactor_#exp_#pub']].head(5).to_dict('index')
def get_interactors(protein: str, interactor_list: str, cluster: str, gene_names: str) -> list:
    interactors = interactor_list.split('\'')[1::2]
    return ([[protein, interactor, cluster, gene_names] for interactor in interactors if interactor >= protein])

all_interactions = df.apply(lambda x: get_interactors(x['MasterAccession'], x['BioGrid_interactor_#exp_#pub'], x['cluster'], x['Gene Names']), axis=1)
all_interactions = pd.DataFrame(reduce(lambda x, y: x + y, all_interactions), columns=['Protein 1', 'Protein 2', 'Cluster', 'Gene Names'])
all_interactions['weight'] = 1.0


In [82]:

all_interactions.to_csv('../data/mRNP_chrono/mRNP_interactions.csv', index=False, sep='\t', header=True)
for cluster in all_interactions['Cluster'].unique():
    all_interactions[all_interactions['Cluster'] == cluster][['Protein 1', 'Protein 2', 'weight']].to_csv(f'../data/mRNP_chrono/mRNP_interactions_{cluster}.sif', index=False, sep='\t', header=False)
    